In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

# Using spark-submit With the Splice Machine Native Spark DataSource

In the basic development course, you developed native spark applications in Jupyter notebooks. In this notebook, we'll develop code that runs on the cluster via *spark-submit*.

## Using the Splice Machine Native Spark DataSource

The *Splice Machine Native Spark DataSource*, which is also referred to as the Spark Adapter, allows you to directly connect Spark DataFrames and Splice Machine database tables, bypassing the need to send your data to/from Spark over a JDBC or ODBC connection. You can efficiently insert, upsert, select, update, and delete data in your Splice Machine tables directly from Spark in a transactionally consistent manner. With the Spark Adapter, transfers of data between Spark and your database are completed without serialization/deserialization, which generates tremendous performance boosts over traditional over-the-wire (sequentially over a JDBC/ODBC connection) transfers.

To use the Spark Adapter in your code, you simply instantiate a `SplicemachineContext` object in your Spark code. You can run Spark applications that interface with your Splice Machine database interactively in the Spark shell, in Jupyter notebooks, or by using our Spark Submit script. One common use of the Adapter is to ingest data into your Splice Machine database directly from a Spark DataFrame.

The Native DataSource allows data scientists to bypass the limitations of the SQL-based JDBC interface in favor of the more scalable and powerful Spark DataFrame API, making it possible for them to operate on data at scale and ingest real-time streaming data with outstanding performance. You can craft applications that use Spark and our Native Spark DataSource in Scala, Python, and Java. Note that you can use the Native Spark DataSource in the Splice Machine ML Manager and Jupyter Notebook interfaces.



## Why Use the Native Data Source?

The primary reason for using the Native DataSource is that it provides dramatic performance improvements for large scale data operations; this is because the DataSource works directly on native DataFrames and RDDs, thus eliminating the need to serialize data. Spark is optimized to work on DataFrames, which is a distributed collection of data (an RDD) organized into named columns, with a schema that specifies data types, that is designed to support efficiently operating on scalable, massive datasets.

The Splice Machine DataSource is native to Spark, which means that it operates directly on these DataFrames and in the same Spark executors that your programs are using to analyze or transform the data. Instead of accessing, inserting, or manipulating data one record at a time over a serialized connection, you can use the Splice Machine Native Spark DataSource to pull the contents of an entire DataFrame into your database, and to pull database query results into a DataFrame.

Splice Machine has observed 100x performance increases compared to using JDBC for operations such as inserting millions of records in a database! For example, a typical web application might use a JDBC connection to query the database, pulling information out one record at a time to populate the screen. The results of each query are serialized (turned into a string of data), then sent over a network connection to the app, and then displayed on the customer’s screen.

In contrast, when you use the Splice Machine Native Spark DataSource, the contents of the database table are typically sitting in a DataFrame in memory that resides on the same Spark executor that’s performing the query. The query takes place in memory, and there’s no need to serialize or stream the results over a wire. Similarly, when the app sends updates to the database, the data is inserted into the database from in-memory DataFrames directly to the tables without serialization. As a result, a great deal of overhead is eliminated, and performance gains can be remarkable.



## The SpliceMachineContext Class

The first thing you need to do when using the Native Spark DataSource is to create an instance of the `SplicemachineContext` class; this is the primary serializable class that you can broadcast in your Spark applications. This class interacts with your Splice Machine cluster in your Spark executors, and provides the methods that you can use to perform operations such as:

* Interfacing with Splice Machine RDD
* Running inserts, updates and deletes on data
* Converting data types between Splice Machine and Spark

## Set Up Maven Coordinates

You'll need to set up a dependency resolution system to be able to build the code. Splice Machine uses Maven, and we've provide sample code and a `pom.xml` file in a public github repo. You can download the source files here: [Sample Source Code](https://github.com/splicemachine/splice-training/tree/master/spark_adapter_example).

You can use this sample as a template for Maven setup. It has both CDH or HWX dependency profiles that can be selected via maven configuration at build time.



## Create SpliceMachineContext

The following code snippet illustrates creating a `SpliceMachineContext` object in Java after setting up the project:

<pre><code>
import com.splicemachine.derby.impl.SpliceSpark;
import com.splicemachine.spark.splicemachine.SplicemachineContext;
import org.apache.spark.SparkConf;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SparkSession;
....

public class TestSpliceMachineContext
{
    public static void main(String[] args) throws Exception 
    {
        String dbUrl = "jdbc:splice://SPLICESERVERHOST:1527/splicedb;user=&lt;yourUserId&gt;;password=&lt;yourPassword&gt;";
        SparkConf conf = new SparkConf();
        SparkSession spark = SparkSession.builder().appName("NativeSparktutor").config(conf).getOrCreate();
        SpliceSpark.setContext(spark.sparkContext());
        SplicemachineContext splicemachineContext = new SplicemachineContext(dbUrl);
        
        //use context for DB operations next...
    }
}
</code></pre>

## Querying with SpliceMachineContext

You can use the `SpliceMachineContext` to obtain access to Spark DataFrame while querying the DB. 

```
import com.splicemachine.derby.impl.SpliceSpark;
import com.splicemachine.spark.splicemachine.SplicemachineContext;
import org.apache.spark.SparkConf;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SparkSession;
....

public class TestSpliceMachineContext
{
    public static void main(String[] args) throws Exception 
    {
        String dbUrl = "jdbc:splice://SPLICESERVERHOST:1527/splicedb;user=&lt;yourUserId&gt;;password=&lt;yourPassword&gt;";
        SparkConf conf = new SparkConf();
        SparkSession spark = SparkSession.builder().appName("NativeSparktutor").config(conf).getOrCreate();
        SpliceSpark.setContext(spark.sparkContext());
        SplicemachineContext splicemachineContext = new SplicemachineContext(dbUrl);
        
        //Query table, use df operations based out of SpliceMachineContext
        String spliceQuery = "select count(*) from " + &lt;TABLE_NAME&gt;;
        splicemachineContext.df(spliceQuery).show();
    }
}
```

## Loading Data with SpliceMachineContext

You can use the `SpliceMachineContext` to load data into a table; in the following example, we read from a parquet file and load the 
data into Splice Machine table.

```
import com.splicemachine.derby.impl.SpliceSpark;
import com.splicemachine.spark.splicemachine.SplicemachineContext;
import org.apache.spark.SparkConf;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SparkSession;
....

public class TestSpliceMachineContext
{
    public static void main(String[] args) throws Exception 
    {
        String dbUrl = "jdbc:splice://SPLICESERVERHOST:1527/splicedb;user=&lt;yourUserId&gt;;password=&lt;yourPassword&gt;";
        SparkConf conf = new SparkConf();
        SparkSession spark = SparkSession.builder().appName("NativeSparktutor").config(conf).getOrCreate();
        SpliceSpark.setContext(spark.sparkContext());
        SplicemachineContext splicemachineContext = new SplicemachineContext(dbUrl);
        
        //create sample dataset
        List&lt;Row&gt; cars = new ArrayList<Row>();
    	StructType schema = DataTypes.createStructType(new StructField[] {
                DataTypes.createStructField("SERIAL",  DataTypes.IntegerType, true),
                DataTypes.createStructField("MAKE", DataTypes.StringType, true),
                DataTypes.createStructField("MODEL", DataTypes.StringType, true)
        });
    	
    	cars.add(RowFactory.create(1, "Toyota", "Camry"));
    	cars.add(RowFactory.create(2, "Honda", "Accord"));
    	cars.add(RowFactory.create(3, "Subaru", "Impreza"));
    	cars.add(RowFactory.create(4, "Chevy", "Volt"));
    	
    	
    	Dataset<Row> carsDF = spark.createDataFrame(cars, schema);
    	carsDF.show();
        
        //Load data into a table using Parquet file
        System.out.println("start create table ...");
    	splicemachineContext.createTable("test.car", schema, 
    			JavaConverters.asScalaIteratorConverter(Arrays.asList("PRIMARY KEY (SERIAL)").iterator()).asScala().toSeq(), "");
    	System.out.println("done create table ... ");
    	
    	StructType outputSchema = splicemachineContext.getSchema("test.car");
    	outputSchema.printTreeString();
    	
    	System.out.println("start insert ...");
    	splicemachineContext.insert(carsDF, "test.car");
    	System.out.println("done insert ... ");
    	
    	System.out.println("start select ...");
    	Dataset&lt;Row&gt; selectedCarsDF = splicemachineContext.df("select * from test.car");
    	System.out.println("done select ...");
    	
    	selectedCarsDF.count();
    	selectedCarsDF.show();
    }
}
```

## Internal Access While Using SpliceMachineContext

By default, Native Spark DataSource queries execute in the Spark application, which is highly performant and allows access to almost all Splice Machine features. However, when your Native Spark DataSource application uses our Access Control List (ACL) feature, there is a restriction with regard to checking permissions.

The specific problem is that the Native Spark DataSource does not have the ability to check permissions at the view level or column level; instead, it checks permissions on the base table. This means that if your Native Spark DataSource application doesn’t have access to the table underlying a view or column, it will not have access to that view or column; as a result, a query against the view or colunn fails and throws an exception.

The workaround for this problem is to tell the Native Spark DataSource to use internal access to the database; this enables view/column permission checking, at a very slight cost in performance. With internal access, the adapter runs queries in Splice Machine and temporarily persists data in HDFS while running the query.

The ACL feature is enabled by setting `splice.authentication.token.enabled=true`.

In addition, to use the Splice Native Spark DataSource, a user must have `execute` permission on the following four system procedures:

* `SYSCS_HBASE_OPERATION`
* `SYSCS_HDFS_OPERATION`
* `SYSCS_GET_SPLICE_TOKEN`
* `SYSCS_CANCEL_SPLICE_TOKEN`

<p class="noteIcon">These procedures are all Splice Machine system procedures that are used internally to efficiently perform direct HBASE and HDFS operations. They <em>are not documented</em> because they are intended only for use by the Splice Machine code itself; however, the Native Spark DataSource uses these procedures, so any user of the Adapter must have permission to execute them.</p>

Here's an example of granting `execute` permission to these procedures to a user named `myUserName`:

```
splice> grant execute on procedure SYSCS_UTIL.SYSCS_HBASE_OPERATION to myUserName;
0 rows inserted/updated/deleted
splice> grant execute on procedure SYSCS_UTIL.SYSCS_HDFS_OPERATION to myUserName;
0 rows inserted/updated/deleted
splice> grant execute on procedure SYSCS_UTIL.SYSCS_GET_SPLICE_TOKEN to myUserName;
0 rows inserted/updated/deleted
splice> grant execute on procedure SYSCS_UTIL.SYSCS_CANCEL_SPLICE_TOKEN to myUserName;
0 rows inserted/updated/deleted
```

Here is an example to use internalDF instead of Spark DF:

<pre><code>
import com.splicemachine.derby.impl.SpliceSpark;
import com.splicemachine.spark.splicemachine.SplicemachineContext;
import org.apache.spark.SparkConf;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SparkSession;
....

public class TestSpliceMachineContext
{
    public static void main(String[] args) throws Exception 
    {
        String dbUrl = "jdbc:splice://SPLICESERVERHOST:1527/splicedb;user=&lt;yourUserId&gt;;password=&lt;yourPassword&gt;";
        SparkConf conf = new SparkConf();
        SparkSession spark = SparkSession.builder().appName("NativeSparktutor").config(conf).getOrCreate();
        SpliceSpark.setContext(spark.sparkContext());
        SplicemachineContext splicemachineContext = new SplicemachineContext(dbUrl);
        
        //Query table, use df operations based out of SpliceMachineContext
        String spliceQuery = "select count(*) from " + &lt;TABLE_NAME&gt;;
        splicemachineContext.internalDf(spliceQuery).show();
    }
}
</code></pre>

## Build and Deploy The Application

You can build the Jar (we suggest building an uber jar) using `maven`, similar to how you build and deploy a typical Spark application. Given the sample pom.xml file in this tutorial, you may have to select an appropriate maven profile when packaging the jar file.

```
mvn clean package
```

Once the jar is built (typically in the `target` folder within the project folder), you can copy it over to the edge node or region server on the cluster to execute with the spark-submit command.

Note that if you are using the Docker image, you can copy the jar over to the running instance with the following command:

```
docker cp <PATH_ON_HOST_MACHINE> spliceserver:<PATH_TO_TARGET_FOLDER_ON_RUNNING_INSTANCE>
```


## Using spark submit to Launch Spark Adapter Applications

You can launch your application with the following command on the provided docker image:

```
spark-submit --conf "spark.dynamicAllocation.enabled=false" --conf "spark.task.maxFailures=2" --conf "spark.driver.memory=512m" --conf "spark.driver.cores=1" --conf "spark.kryoserializer.buffer=512" --conf "spark.kryoserializer.buffer.max=1024" --conf "spark.io.compression.codec=org.apache.spark.io.SnappyCompressionCodec" --conf "spark.executor.extraClassPath=/opt/hadoop/conf/:/opt/spark/jars/*" --conf "spark.driver.extraClassPath=/opt/hadoop/conf/:/opt/spark/jars/*" --class <FULLY_QUALIFIED_EXECUTABLE_CLASS_NAME> --master local[1] --deploy-mode client --num-executors 1 --executor-memory 1G --executor-cores 1 <JAR_FILE_NAME> localhost 1527 splice admin
```

Note that  various executor settings in the above example are just illustrative and need to be be configured for your job and its execution environment.


## For Further Exploration and Understanding

To gain a better understanding of using statistics, try using `spark-submit` by building a JAR file and running these exercises:

1. Use Spark Adapter to create a set of tables where you can exercise some queries including join operations.

2. Use the tables in Step-1 to insert data into tables with Spark Adapter. Use both DF and internalDF to do these operations.

3. Analyze the table using Spark Adapter (NOTE: it may not execute due to memory constraints on docker image on localhost, skip this step in that case)

4. Query the table and verify data load. 

5. Use a Join operation via Spark Adapter

6. Drop the table(s)


## Where to Go Next
To finish this class, please complete the exercises in the <a href="./j.%20Exercises.ipynb"><em>Exercises for This Class</em></a> notebook, which test your understanding of the material we've covered.</td>
